- data source: http://www.manythings.org/anki/

- This file also works. Now using pre-trained embeddings.
- Used functions for code blocks. But that didn't really make things much neater. 
- Bidirectional LSTM for encoder

In [1]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, RNN, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import string 
import regex as re

from sklearn.model_selection import train_test_split

import io
import spacy

import warnings 
warnings.filterwarnings('ignore')

# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [68]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# df_en_de = pd.read_table('/content/gdrive/MyDrive/deu-eng/deu.txt', names=['eng', 'deu', 'attr'])

In [2]:
df_en_de = pd.read_table('deu-eng/deu.txt', names=['eng', 'deu', 'attr'])

In [3]:
df_en_de = df_en_de.drop('attr',axis = 1).rename(columns = {'eng':'english', 'deu':'german'})

In [4]:
# Lowercase all characters
df_en_de['english'] = df_en_de['english'].apply(lambda x: x.lower())
df_en_de['german'] = df_en_de['german'].apply(lambda x: x.lower())

# Remove quotes
df_en_de['english'] = df_en_de['english'].apply(lambda x: re.sub("'", '', x))
df_en_de['german'] = df_en_de['german'].apply(lambda x: re.sub("'", '', x))

# Convert umlauts and sharp s:
df_en_de['german'] = df_en_de['german'].apply(
                            lambda x: x.replace('ü', 'ue').replace('ä', 'ae').replace('ö', 'oe').replace('ß', 'ss')
                            )

# Set of all special characters
exclude = set(string.punctuation) 

# Remove all the special characters
df_en_de['english'] = df_en_de['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df_en_de['german']=df_en_de['german'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Add start and end tokens to target sequences
df_en_de['german'] = df_en_de['german'].apply(lambda x : 'START_ '+ x + ' _END')

In [5]:
#rename dataframe for convenience
pairs = df_en_de
max_len = 10

pairs = df_en_de
pairs['english_length'] = pairs['english'].apply(lambda x: len(x.split(' ')))
pairs['german_length'] = pairs['german'].apply(lambda x: len(x.split(' ')))

# remove ascii symbols
pairs['english_cleaned'] = pairs['english'].apply(lambda x: x.encode("ascii", "ignore"))
pairs['english_cleaned'] = pairs['english_cleaned'].apply(lambda x: x.decode())

# remove ascii symbols
pairs['german_cleaned'] = pairs['german'].apply(lambda x: x.encode("ascii", "ignore"))
pairs['german_cleaned'] = pairs['german_cleaned'].apply(lambda x: x.decode())

pairs = pairs[pairs['english_length'] <= max_len]
pairs = pairs[pairs['german_length'] <= max_len]

# Reduced dataset to check if model works
pairs = pairs.sample(frac = 0.1, random_state = 1)

In [ ]:
!python -m spacy download en_core_web_lg
import en_core_web_lg

!python -m spacy download de_core_news_sm
import de_core_news_sm

In [7]:
text_source = pairs['english_cleaned']
text_target = pairs['german_cleaned']

In [8]:
nlp_source = en_core_web_lg.load()
nlp_target = de_core_news_sm.load()

In [9]:
Vectorizer_source = TextVectorization()
Vectorizer_target = TextVectorization()

Vectorizer_source.adapt(text_source)
Vectorizer_target.adapt(text_target)

vocab_source = Vectorizer_source.get_vocabulary()
vocab_target = Vectorizer_target.get_vocabulary()

2022-08-01 16:48:10.835211: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# create list of words
vocab_source = [str(word) for word in vocab_source]
vocab_target = [str(word) for word in vocab_target]

# remove empty string from vocab
vocab_source.remove('')
vocab_target.remove('')

# The embeddings downloaded from spacy don't include our 'START_' and '_END' tokens
# Add them to "vocab_target"
vocab_target.append('START_')
vocab_target.append('_END')

In [12]:
# vocab size for source and target

vocab_len_source = len(vocab_source)
vocab_len_target = len (vocab_target)

vocab_len_source, vocab_len_target

(6446, 9992)

In [13]:
#generate the embedding matrix for source vocab

# add 1 for zero padding (for Embedding layer)
num_tokens_source = len(vocab_source)
num_tokens_source = num_tokens_source + 1

# source language embedding dimensions
embedding_dim_source = len(nlp_source('The').vector)
# initialise embedding matrix for source language
embedding_matrix_source = np.zeros((num_tokens_source, embedding_dim_source))

# word-to-index and index-to-word mappings for source language
word_idx_source = {}
idx_word_source = {}
# notice we start indexing from 1 (no word is assigned to 0 index)
for i, word in enumerate(vocab_source):
    embedding_matrix_source[i+1] = nlp_source(word).vector      # load vectors into embedding matrix
    word_idx_source[word] = int(i+1)                            # word-to-index map
    idx_word_source[i+1] = word                                 # index-to-word map

# generate the embedding matrix for target vocab

# add 1 for zero padding (for Embedding layer)
num_tokens_target = len(vocab_target)
num_tokens_target = num_tokens_target + 1

# target language embedding dimensions
embedding_dim_target = len(nlp_target('Der').vector)
# initialise embedding matrix for target language
embedding_matrix_target = np.zeros((num_tokens_target, embedding_dim_target))

# word-to-index and index-to-word mappings for target language
word_idx_target = {}
idx_word_target = {}
for i, word in enumerate(vocab_target):
    # iterate over all words excluding the final two ("START_" and "_END")
    if i < len(vocab_target)-2: 
        embedding_matrix_target[i+1] = nlp_target(word).vector      # load vectors into embedding matrix
        word_idx_target[word] = int(i+1)                            # word-to-index map
        idx_word_target[i+1] = word                                 # index-to-word map
    if word == 'START_':
        # assign embedding vector with random values for "START_" token 
        embedding_matrix_target[i+1] = np.random.rand((embedding_dim_target))
        word_idx_target[word] = int(i+1)
        idx_word_target[i+1] = word
    if word == '_END':
        # assign embedding vector with random values for "_END" token
        embedding_matrix_target[i+1] = np.random.rand((embedding_dim_target))
        word_idx_target[word] = int(i+1)
        idx_word_target[i+1] = word

- run time for small dataset: 1 m 4s

In [15]:
num_tokens_source, len(vocab_source)
num_tokens_target, len(vocab_target)

(9993, 9992)

In [16]:
# Make sure to load CLEANED data
X, y = pairs['english_cleaned'], pairs['german_cleaned']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=101)

In [17]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        # for every batch j
        for j in range(0, len(X), batch_size):          
            encoder_input_data = np.zeros((batch_size, max_len), dtype='float32')         # (batch_size, max_len)      
            decoder_input_data = np.zeros((batch_size, max_len), dtype='float32')         # (batch_size, max_len)
            decoder_target_data = np.zeros((batch_size, max_len, num_tokens_target), dtype='float32') # (batch_size, max_len, num_tokens_target)
            
            # for every example i
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    # for every time-step t, insert index for encoder input
                    encoder_input_data[i, t] = word_idx_source[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    # for every time-step t, insert index for decoder input (excluding final time-step)
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = word_idx_target[word] # decoder input seq
                    # create one-hot vector for decoder output, excluding the START_ token
                    # offset by one timestep
                    if t>0:
                        decoder_target_data[i, t - 1, word_idx_target[word]] = 1. 
                        
            
            yield([encoder_input_data, decoder_input_data], decoder_target_data)
        

- global variables for one_step_attention:

In [18]:
Tx = max_len
Ty = max_len
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis = -1)
densor1 = Dense(10, activation = 'tanh')
densor2 = Dense (1, activation = 'relu')
dotor = Dot(axes = 1)

In [19]:
def one_step_attention (h, s_prev):

    # calculate the Context vector for one time-step of decoder

    # h = (m, Tx, n_h)
    # s_prev = (m, n_s)
    # returns: context – we will then use [context; y_prev] as input of Decoder

    s_prev = repeator(s_prev)                   # (m, Tx, n_s)
    concat = concatenator([h, s_prev])          # (m, Tx, n_h + n_s)
    e = densor1 (concat)                        # (m, Tx, 10)
    energies = densor2 (e)                      # (m, Tx, 1)
    alphas = tf.nn.softmax(energies, axis = 1)  # (m, Tx, 1)
    context = dotor([alphas, h])                # alphas = (m, Tx, 1)
                                                # h = (m, Tx, n_h)
                                                # (m, 1, n_h)
    return context                              # (m, 1, n_h)

In [30]:
emb_dim_source = embedding_matrix_source.shape[1]
emb_dim_target = embedding_matrix_target.shape[1]

n_h = 200
n_s = 200


- global variables for pre-attention LSTM:

In [31]:
x_inputs = Input(shape = (Tx,))                         # (None, Tx) = (m, Tx)
x_emb_layer = Embedding(num_tokens_source, emb_dim_source, mask_zero = True,
                        embeddings_initializer = Constant(embedding_matrix_source),
                        trainable = False)  

x_emb = x_emb_layer(x_inputs)                         # (None, Tx, x_emb_dim) = (m, Tx, x_emb_dim)

# Use bidirectional LSTM
enc_lstm_layer = Bidirectional(LSTM(n_h, dropout = 0.3, recurrent_dropout = 0.3, return_sequences=True, return_state = True))
h_enc, s_enc, c_enc, _, _ = enc_lstm_layer(x_emb)                             # h_enc = (None, Tx, 2 * n_h) = (m, Tx, 2*n_h)
                                                                


- global variables for post-attention LSTM:

In [32]:
y_inputs = Input(shape = (Ty,))                         # (None, Ty) = (m, Ty)

y_emb_layer = Embedding (num_tokens_target, 
                        emb_dim_target, 
                        mask_zero = True,
                        embeddings_initializer = Constant(embedding_matrix_target),
                        trainable = False)
                        
y_emb = y_emb_layer(y_inputs)                         # (None, Ty, y_emb_dim) = (m, Ty, y_emb_dim)

dec_lstm_layer = LSTM(n_s, dropout = 0.3, recurrent_dropout = 0.3, return_state = True)
output_layer = Dense(num_tokens_target, activation='softmax')



In [33]:
def training_model():

    # s<0> and c<0> for decoder = s<Ty> and c<Ty> for encoder
                                                                 
    s_dec = s_enc
    c_dec = c_enc
    
    outputs = []
    for t in range(Ty):
        context = one_step_attention(h_enc, s_dec)                                            # context = (m, 1, n_h)
        concat = Concatenate(axis = -1)([context, tf.expand_dims(y_emb[:,t,:],1)])            # concat = (m, 1, n_h + y_emb_dim)
        
        # update decoder LSTM hidden state (s) and cell state (c)
        _, s_dec, c_dec = dec_lstm_layer (initial_state = [s_dec, c_dec], inputs = concat)        # s = (None, Ty, n_s)
        
        # pass decoder LSTM hidden state (s) through output layer to get y prediction
        out = output_layer(s_dec)                                                                   # out = (m, num_decoder_tokens)
        outputs.append(out)
                                                                
    outputs = tf.stack(outputs, axis = 1)                                                           # outputs = (m, Ty, num_decoder_tokens)
    training_model = Model(inputs = [x_inputs, y_inputs], outputs = outputs)

    return training_model

In [34]:
training_model = training_model()
training_model.compile(optimizer= 'Adam', loss='categorical_crossentropy', metrics=['acc'])

train_samples = len(X_train) # Total Training samples
val_samples = len(X_test) # total validation samples
batch_size = 128
epochs = 5


In [35]:
history = training_model.fit(generate_batch(), 
                    steps_per_epoch=train_samples//batch_size, 
                    epochs = 5,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples // batch_size, 
                    verbose = 1)

Epoch 1/5
130/130 [==============================] - 66s 340ms/step - loss: 3.9550 - acc: 0.1273 - val_loss: 3.6071 - val_acc: 0.1514
Epoch 2/5
130/130 [==============================] - 41s 316ms/step - loss: 3.3487 - acc: 0.1695 - val_loss: 3.2495 - val_acc: 0.1898
Epoch 3/5
130/130 [==============================] - 40s 306ms/step - loss: 3.0277 - acc: 0.2037 - val_loss: 3.0147 - val_acc: 0.2142
Epoch 4/5
130/130 [==============================] - 40s 312ms/step - loss: 2.7801 - acc: 0.2230 - val_loss: 2.8306 - val_acc: 0.2323
Epoch 5/5
130/130 [==============================] - 42s 319ms/step - loss: 2.5814 - acc: 0.2402 - val_loss: 2.6914 - val_acc: 0.2467


In [54]:
hist_df = pd.DataFrame(history.history)
hist_df

,loss,acc,val_loss,val_acc
0,1.514828,0.332356,2.134959,0.311377
1,1.454318,0.339177,2.110950,0.315015
2,1.399507,0.346094,2.092885,0.317871
3,1.346242,0.355060,2.074517,0.320654
4,1.300279,0.361382,2.054834,0.322803
5,1.251908,0.370853,2.045037,0.325537
6,1.209428,0.379495,2.033970,0.327197
7,1.169399,0.386148,2.017985,0.330762
8,1.132447,0.394736,2.005462,0.330737
9,1.095821,0.402019,1.992962,0.333887


In [58]:
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f, index=False)

In [59]:
pd.read_csv('history.csv')

,loss,acc,val_loss,val_acc
0,1.514828,0.332356,2.134959,0.311377
1,1.454318,0.339177,2.110950,0.315015
2,1.399507,0.346094,2.092885,0.317871
3,1.346242,0.355060,2.074517,0.320654
4,1.300279,0.361382,2.054834,0.322803
5,1.251908,0.370853,2.045037,0.325537
6,1.209428,0.379495,2.033970,0.327197
7,1.169399,0.386148,2.017985,0.330762
8,1.132447,0.394736,2.005462,0.330737
9,1.095821,0.402019,1.992962,0.333887


In [60]:
def encoder_model():
    # x_inputs defined above as Input
    # h_enc, s_enc, c_enc also defined above
    encoder_model = Model(inputs = x_inputs, outputs = [h_enc, s_enc, c_enc])
    return encoder_model

def decoder_model():
    # The below three decoder inputs will come from encoder_model.predict()
    decoder_input_h = Input(shape = (Tx, 2*n_h))               # (None, Tx, 2*n_h) 
    decoder_input_s = Input(shape=(n_h, ))                    # (None, n_h) 
    decoder_input_c = Input(shape = (n_h,))                   # (None, n_h)

    # y_emb_2 will be our y_pred at t-1
    y_inp_2 = Input(shape = (None,))                        # (None, None) = (m, Ty)
    y_emb_2 = y_emb_layer(y_inp_2)                          # (None, None, y_emb_dim) = (m, Ty, y_emb_dim)

    # Use decoder_input_s and decoder_input_h to compute context vector
    context = one_step_attention(decoder_input_h, decoder_input_s)    # (m, 1, 2*n_h)

    # concatenate context with y_emb_2
    concat2 = Concatenate(axis = -1)([context, tf.expand_dims(y_emb_2[:,-1,:],1)])                       
                                                            # concat2 = (None, 1, 2*n_h + y_emb_dim)

    # Feed concat2 as input; decoder_input_s and decoder_input_c as initial state
    _, decoder_output_s, decoder_output_c = dec_lstm_layer (
                                                            initial_state = [decoder_input_s, decoder_input_c], 
                                                            inputs = concat2
                                                            )     
                                                # decoder_output_s = (None, n_s) 
                                                # decoder_output_c = (None, n_s) 


    decoder_output_y = output_layer(tf.expand_dims(decoder_output_s,1))           # (None, 1, num_decoder_tokens)

    decoder_model = Model(inputs = [decoder_input_h, decoder_input_s, decoder_input_c, y_inp_2],
                            outputs = [decoder_output_y, decoder_output_s, decoder_output_c])
    return decoder_model

In [61]:
encoder_model = encoder_model()
decoder_model = decoder_model()

In [62]:
def decode_sequence(input_sequence):

    # input_sequence = (1, max_len)
    
    # get hidden states + final hidden state + final cell state from encoder 
    h_enc_pred, s_enc_pred, c_enc_pred = encoder_model.predict(input_sequence)
    
    # define y_pred at time 0    
    target_seq = np.zeros((1,1))
    target_seq[0,0] = word_idx_target['START_']


    stop_condition = False
    decoded_sentence = ''

    # initialise hidden state and cell state input for decoder
    decoder_s_pred = s_enc_pred                         # (None, n_h) = (m, n_h)
    decoder_c_pred = c_enc_pred                         # (None, n_h) = (m, n_h)
    
    while not stop_condition:
        decoder_y_pred, decoder_s_pred, decoder_c_pred = decoder_model.predict([h_enc_pred, decoder_s_pred, decoder_c_pred, target_seq])  
        y_index = np.argmax(decoder_y_pred[0,-1,:])
        y_word = idx_word_target[y_index]
        decoded_sentence += ' ' + y_word

        # Exit condition: either hit max length
        # or find stop character.
        if (y_word == '_END' or
           len(decoded_sentence.split()) > max_len):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = y_index
        
        
        
    return decoded_sentence


In [63]:
def sentence_to_seq(sentence):
    """
    sentence = string
    """
    
    encoder_input_data = np.zeros((1, max_len))     # (1, max_len) 
    sentence = sentence.lower().split()
    for j, word in enumerate(sentence):
        encoder_input_data[0,j] = word_idx_source[word]        # (1, max_len)
    return encoder_input_data                           # (1, max_len)


In [40]:
sentences = list(X_train.iloc[:10].values)          # list of sentences

translations = []
for sentence in sentences:
    seq = sentence_to_seq(sentence)             # seq = (1, max_len)
    #print(seq, seq.shape)
    translation = decode_sequence(seq)
    translations.append(translation)

sentence_translation_pairs = zip (sentences, translations)
for elem in sentence_translation_pairs:
    print (elem)

('why didnt tom visit boston', ' warum tom nicht tom und maria _END')
('please dont waste electricity', ' bitte bitte nicht _END')
('tom insulted the waiter', ' tom hat die tuer _END')
('theres nothing to worry about', ' das ist nicht zu sein _END')
('they appointed him manager', ' sie hat ihn zu hause _END')
('one more bottle of wine please', ' das buch fuer eine zeit _END')
('do you know what tom said about it', ' warum du tom dass tom hat tom _END')
('tom is always making me angry', ' tom ist mir sehr nie _END')
('i want to speak with tom please', ' ich moechte tom zu helfen _END')
('arent you pushing it too far', ' du nicht so so nicht _END')


In [46]:
sentences = list(X_train.iloc[:10].values)          # list of sentences

translations = []
for sentence in sentences:
    seq = sentence_to_seq(sentence)             # seq = (1, max_len)
    #print(seq, seq.shape)
    translation = decode_sequence(seq)
    translations.append(translation)

sentence_translation_pairs = zip (sentences, translations)
for elem in sentence_translation_pairs:
    print (elem)

('why didnt tom visit boston', ' warum tom nicht nach boston _END')
('please dont waste electricity', ' bitte bitte nicht die _END')
('tom insulted the waiter', ' tom hat die haare gesehen _END')
('theres nothing to worry about', ' es ist nichts zu essen zu tun _END')
('they appointed him manager', ' sie haben ihn zu heiraten _END')
('one more bottle of wine please', ' ein paar kaffee bitte _END')
('do you know what tom said about it', ' weisst du dass tom was tom gesagt _END')
('tom is always making me angry', ' tom ist immer immer mich _END')
('i want to speak with tom please', ' ich will tom zu helfen _END')
('arent you pushing it too far', ' bist du nicht so schnell _END')


In [52]:
sentences = list(X_train.iloc[:10].values)          # list of sentences

translations = []
for sentence in sentences:
    seq = sentence_to_seq(sentence)             # seq = (1, max_len)
    #print(seq, seq.shape)
    translation = decode_sequence(seq)
    translations.append(translation)

sentence_translation_pairs = zip (sentences, translations)
for elem in sentence_translation_pairs:
    print (elem)

('why didnt tom visit boston', ' warum tom nicht nach boston boston _END')
('please dont waste electricity', ' bitte haben keine sorgen _END')
('tom insulted the waiter', ' tom hat die haare _END')
('theres nothing to worry about', ' es gibt es nichts zu reden _END')
('they appointed him manager', ' sie haben ihn angelogen _END')
('one more bottle of wine please', ' eine tasse bitte wein _END')
('do you know what tom said about it', ' weisst du was tom gesagt hat _END')
('tom is always making me angry', ' tom ist immer immer mich wuetend _END')
('i want to speak with tom please', ' ich will mit tom helfen _END')
('arent you pushing it too far', ' bist du es nicht so schnell _END')


In [64]:
sentences = list(X_train.iloc[:10].values)          # list of sentences

translations = []
for sentence in sentences:
    seq = sentence_to_seq(sentence)             # seq = (1, max_len)
    #print(seq, seq.shape)
    translation = decode_sequence(seq)
    translations.append(translation)

sentence_translation_pairs = zip (sentences, translations)
for elem in sentence_translation_pairs:
    print (elem)

('why didnt tom visit boston', ' warum ist tom boston nicht nach boston _END')
('please dont waste electricity', ' bitte mache keine unnoetigen _END')
('tom insulted the waiter', ' tom hat den kellner gemacht _END')
('theres nothing to worry about', ' es gibt nichts zu loesen _END')
('they appointed him manager', ' sie ernannten ihn angelogen _END')
('one more bottle of wine please', ' eine tasse kaffee bitte wein _END')
('do you know what tom said about it', ' weisst du was tom darueber erzaehlt _END')
('tom is always making me angry', ' tom macht immer immer wuetend _END')
('i want to speak with tom please', ' ich will mit tom helfen _END')
('arent you pushing it too far', ' bist du nicht so schnell _END')
